In [2]:
import cv2
import numpy as np
import csv
import sys
import os
import math

Process images from blender and store the data (combined with the correct position data from the text file) in a csv file

In [3]:
# Function to display progress in the console
def update_progress(progress):
    bar_length = 50  # Total length of the progress bar
    filled_length = int(bar_length * progress / 100)
    bar = '#' * filled_length + '_' * (bar_length - filled_length)
    sys.stdout.write('\r[{0}] {1}%'.format(bar, progress))
    sys.stdout.flush()

Each image represents a unique possible position of the game object, in the database we describe each image with 4 values: center_pixel_x, center_pixel_y, rect_width, rect_height - Symmetrical Object (Algae, Note)

In [ ]:
# Function to find red regions and extract bounding rectangle
def find_bounding_rect(image):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range of red color in HSV
    lower_red = np.array([0, 20, 25])
    upper_red = np.array([30, 255, 255])
    
    # Combine masks to get only red pixels
    red_mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Find contours
    contours, _ = cv2.findContours(red_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    
    # Get bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)
    cv2.imshow("Frame", image)
    cv2.waitKey(1)

    return (x, y, w, h)

Each image represents a unique possible position of the game object, in the database we describe each image with 5 values: center_pixel_x, center_pixel_y, rect_width, rect_height, angle - Non Symmetrical Object (Cone, Coral)

In [ ]:
# Works by finding the two longest edges of the contour
# and averaging their angles
# Used for 2025 Coral
def find_angle(contour, img=None, draw=True):
    """
    Given a contour, find the two longest straight lines,
    average their directions, and return the dominant angle in degrees.
    """
    if len(contour) < 2:
        return None

    # 1. Simplify contour (remove small jitter)
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # 2. Collect line segments and their angles
    lines = []
    for i in range(len(approx)):
        p1 = approx[i][0]
        p2 = approx[(i + 1) % len(approx)][0]  # wrap around
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        length = math.hypot(dx, dy)
        if length > 2:  # ignore tiny edges
            angle = math.degrees(math.atan2(dy, dx))
            lines.append((length, angle, p1, p2))

    if len(lines) < 2:
        return None

    # 3. Take two longest lines
    lines.sort(reverse=True, key=lambda x: x[0])
    longest = lines[:2]

    # 4. Compute average direction
    # Handle circular mean (avoid averaging 179° and -179° to get 0°)
    angles = [math.radians(l[1]) for l in longest]
    x_mean = np.mean([math.cos(a) for a in angles])
    y_mean = np.mean([math.sin(a) for a in angles])
    avg_angle = math.degrees(math.atan2(y_mean, x_mean))

    avg_angle = (avg_angle + 360 + 90) % 180  # normalize to [0, 180)

    # 5. Optional drawing
    if img is not None and draw:
        for _, angle, p1, p2 in longest:
            cv2.line(img, tuple(p1), tuple(p2), (0, 255, 0), 2)
        # Draw the averaged orientation line at the contour center
        cx, cy = np.mean(approx[:, 0, :], axis=0).astype(int)
        length = 50
        x2 = int(cx + length * math.cos(math.radians(avg_angle)))
        y2 = int(cy + length * math.sin(math.radians(avg_angle)))
        cv2.arrowedLine(img, (cx, cy), (x2, y2), (255, 255, 0), 2, tipLength=0.2)

    return avg_angle

In [ ]:
def find_bounding_rect_angle(image):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range of red color in HSV
    lower_red = np.array([0, 20, 25])
    upper_red = np.array([30, 255, 255])
    
    # Combine masks to get only red pixels
    red_mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Find contours
    contours, _ = cv2.findContours(red_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)

    # Get bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)
    angle = find_angle(contour, image, draw=True)
    
    cv2.imshow("Frame", image)
    cv2.waitKey(1)
    
    return (x + w / 2, y + h / 2), (w, h), angle

### Load data from Blender

In [ ]:
# Image folder path and CSV file path
images_folder = '../Data/Custom-Data/Images'
text_file_path = '../Data/Custom-Data/PositionData.txt'
csv_file_path = '../Data/Custom-Data/FullData.csv'

In [ ]:
# Count .png files in the folder
total_iterations = len([file for file in os.listdir(images_folder) if file.endswith('.jpg')])
print(f"Total images found: {total_iterations}")
# Get a sorted list of image filenames in the folder
image_files = range(0, total_iterations)


position_data = []
with open(text_file_path, 'r') as file:
    for line in file:
        x, y, a = line.strip().split(',')
        
        position_data.append([float(x), float(y), float(a)])

if (len(position_data) != total_iterations):
    print(f"Warning: Number of position data points ({len(position_data)}) does not match number of images ({total_iterations}).")

### Process Data from Blender - Non Symmetrical Object (Cone, Coral)

In [ ]:
targetRes = (1280, 720)

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Image', 'Center_X', 'Center_Y', 'Width', 'Height', 'Image_angle', 'x_position', 'y_position', 'angle'])

    # Loop through all files in the folder
    for filename in image_files:
        # Read image
        image = cv2.imread(os.path.join(images_folder, ("render_"+ str(filename)+ ".jpg")))
        
        image = cv2.resize(image, targetRes, interpolation=cv2.INTER_LINEAR)

        # Find red rectangles
        rect_angle = find_bounding_rect_angle(image)

        # Extract rectangle parameters
        (center_x, center_y), (width, height), angle = rect_angle

        csv_writer.writerow([filename, np.intp(center_x), np.intp(center_y), np.intp(width), np.intp(height), np.intp(angle), position_data[filename][0], position_data[filename][1], position_data[filename][2]])
        
        # Update progress
        progress = int(100 * (filename + 1) / total_iterations)
        update_progress(progress)

cv2.destroyAllWindows()

### Process Data from Blender - Symmetrical Object (Algae, Note)

In [ ]:
targetRes = (1280, 720)

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Image', 'Center_X', 'Center_Y', 'Width', 'Height', 'x_position', 'y_position'])

    # Loop through all files in the folder
    for filename in image_files:
        # Read image
        image = cv2.imread(os.path.join(images_folder, ("render_"+ str(filename)+ ".jpg")))
        
        image = cv2.resize(image, targetRes, interpolation=cv2.INTER_LINEAR)

        # Find red rectangles
        rect = find_bounding_rect(image)
        
        x, y, w, h = rect
        center_x = x + w // 2
        center_y = y + h // 2
        csv_writer.writerow([filename, center_x, center_y, w, h, position_data[filename][0], position_data[filename][1]])
        
        # Update progress
        progress = int(100 * (filename + 1) / total_iterations)
        update_progress(progress)

cv2.destroyAllWindows()